In [10]:
import pandas as pd
import statsmodels.api as sm
import numpy as np





df_markets = pd.read_csv('data/test/markets.csv')


with open('data/test/multi_log_reg/results.txt', 'w') as f:
    for ticker in df_markets['Ticker']:
        df2=pd.read_csv(f'data/test/multi_log_reg/{ticker}.csv')
        df= pd.read_csv(f'data/test/linear/{ticker}.csv')
        df2 = df2.rename(columns={'Date': 'date'})
        
        df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Ensure the conversion worked
        if not np.issubdtype(df['date'].dtype, np.datetime64):
            print("Warning: 'date' column is not in datetime format after conversion.")

    # If datetime conversion was successful, apply timezone conversion
        df['date'] = df['date'].dt.tz_localize('UTC')  # Use .tz_localize() if naive
        df['date'] = df['date'].dt.strftime('%Y-%m-%dT%H:%M:%S%z')

        df2['date'] = pd.to_datetime(df2['date']).dt.strftime('%Y-%m-%dT%H:%M:%S%z')
        df2_subset = df2[['date', 'effect']]

        # Merge df and df2_subset on the Date column
        merged_df = pd.merge(df, df2_subset, on='date', how='left')
    
    
        #This part does the regreesion modeling
        X = sm.add_constant(merged_df[['num', 'max', 'min_dist','tsunami']])
        y = merged_df['effect']
        

        # Fit the regression model
        # Fit the logistic regression model
        mlogit_model = sm.MNLogit(y, X)
        result = mlogit_model.fit()
        
        # Write the ticker name and model summary to file
        f.write(f"Ticker: {ticker}\n")
        f.write(result.summary().as_text())
        f.write("\n" + "="*80 + "\n")



C:\Users\SJANO\AppData\Local\Temp\ipykernel_33756\906596529.py:18: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['date'] = pd.to_datetime(df['date'], errors='coerce')


AttributeError: Can only use .dt accessor with datetimelike values